In [1]:
import numpy as np

In [4]:
x = np.array([[1, 2, 3], [4, 3, 1], [6, 3, 1], [6, 1, 0]])
y = np.array([0, 1, 1, 1])
Ck = 1
i = 2

In [5]:
x[y == Ck] - x[i]

array([[-2,  0,  0],
       [ 0,  0,  0],
       [ 0, -2, -1]])

In [6]:
np.apply_along_axis(np.linalg.norm, 1, x[y == Ck] - x[i])

array([ 2.        ,  0.        ,  2.23606798])

In [ ]:
## Задание 1
from sklearn.base import BaseEstimator, ClusterMixin

class Clustering(BaseEstimator, ClusterMixin):
    """
    Implement clustering algorithm according 
    """
    
    #There and everywhere then the euclidian distance is used as a metrics
    def single_linkage(self, C1, C2, X, d = 0):
        labels = self.get_labels()
        ind1 = (labels == C1)
        ind2 = (labels == C2)
        if X[ind1].shape[0] == 0 or X[ind2].shape[0] == 0:
            return np.inf
        return np.square(X[ind1, np.newaxis, :] - X[ind2]).sum(axis=2).min()**(1.0/2)
    
    def nml_single_linkage(self, X):
        C1, C2 = 0
        labels = self.get_labels()
    
    def complete_linkage(self, C1, C2, X):
        labels = self.get_labels()
        ind1 = (labels == C1)
        ind2 = (labels == C2)
        if X[ind1].shape[0] == 0 or X[ind2].shape[0] == 0:
            return np.inf
        return np.square(X[ind1, np.newaxis, :] - X[ind2]).sum(axis=2).max()**(1.0/2)
        
    def average_linking(self, C1, C2, X):
        labels = self.get_labels()
        ind1 = (labels == C1)
        ind2 = (labels == C2)
        if X[ind1].shape[0] == 0 or X[ind2].shape[0] == 0:
            return np.inf
        dist_sum = np.square(X[ind1, np.newaxis, :] - X[ind2]).sum(axis=2)**(1.0/2).sum()
        return dist_sum / ind1.shape[0] / ind2.shape[0]
    
    def get_labels(self):
        return self.labels_seq[self.layer - 1]
    
    def get_least_clusters(self, X):
        tmp_dist = np.inf
        C1, C2 = 0, 0
        for x in range(0, X.shape[0]):
            for y in range(0, X.shape[0]):
                if x < y:
                    tmp = self.linkage(x, y, X)
                    if tmp < tmp_dist:
                        C1, C2 = x, y
                        tmp_dist = tmp
        return C1, C2
                       
    def get_least_clusters_nrm(self):
        tmp = self.dist.argmin()
        return tmp / dist.shape[0], tmp % dist.shape[0]
    
    def fix_dist(C1, C2):
        F[C1] = np.apply_along_axis(self.linkage, axis=1, F[C1], C1)
        F[:, C1] = np.apply_along_axis(self.linkage, axis=1, F[:, C1], C1)
        F[C2] = np.inf
        F[:C2] = np.inf
    
    def __init__(self, **kwargs):
        """
        Please add necessary algoritm parameters to class constructor.
        """
        if kwargs['linkage'] == 'single-linkage':
            self.linkage = self.single_linkage
        elif kwargs['linkage'] == 'complete-linkage':
            self.linkage = self.complete_linkage
        elif kwargs['linkage'] == 'average-linkage':
            self.linkage = self.average_linking
        else:
            self.linkage = self.single_linkage
            
        self.k = kwargs['k']
        
        self.layer = 1
        
        
        return
        
    def fit(self, X, y=None):
        """
        Use data matrix x to compute model parameters
        """
        self.labels_seq = np.empty([X.shape[0] - self.k + 1, X.shape[0]], dtype=np.int)
        
        self.labels_seq[0] = [x for x in xrange(0, X.shape[0])]
        
        for ind in range(1, X.shape[0] - self.k + 1):
            #print ind
            C1, C2 = self.get_least_clusters(X)
            C1, C2 = min(C1, C2), max(C1, C2)
            self.labels_seq[ind] = self.labels_seq[ind - 1]
            self.labels_seq[ind, self.labels_seq[ind] == C2] = C1
            self.layer += 1
            self.fix_dist(C1, C2)
        
        #self.x_len = len(x)
        return self
    
    def predict(self, X):
        #return labels for dividion into self.k clusters
        return self.labels_seq[X.shape[0] - self.k]
    
    def fit_predict(self, X, y=None):
        self.fit(X, y)
        return self.predict(X)